In [7]:
! pip install openpyxl datetime pandas

In [8]:
import os
from openpyxl import Workbook
from datetime import datetime
import pandas as pd

In [9]:
user_input = 'prism_n1_le23_d202.log' # Enter path to log here
assert os.path.exists(user_input), "I did not find the file at "+str(user_input)
f = open(user_input,'r')
print("Log File found, beginning analysis...")
f.close()

Log File found, beginning analysis...


In [10]:
ProposerBlockMined = 'ProposerBlockMined'
ValidProposerBlockDownloaded = 'ValidProposerBlockDownloaded'
ProposerBlockConfirmed = 'ProposerBlockConfirmed'
ProposerBlockRolledBack = 'ProposerBlockRolledBack'
epoch = datetime.utcfromtimestamp(0)

hashToNumber = {}
hashToCreationTimestamp = {}
hashToCreationEvent = {}
hashToConfirmationTimestamp = {}
hashToRollbackTimestamp = {}

In [11]:
def getTimestamp(timestr):
    dateTimeObj = datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S,%f')
    delta = dateTimeObj - epoch
    return delta.total_seconds()*1000

In [12]:
with open(user_input, 'r') as f:
    for line in f:
        if "BENCHMARKING:" not in line:
            continue
        benchmarkSubstrings = line[line.find("BENCHMARKING"):-1].split(':')
        dateTimeString = line.split('[')[0][:-1]
        timestamp = getTimestamp(dateTimeString)
        event = benchmarkSubstrings[1]
        blockNumber = benchmarkSubstrings[3]
        blockHash = benchmarkSubstrings[-1][1:-1]
        if event == ProposerBlockMined or event == ValidProposerBlockDownloaded:
            if hashToCreationTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToCreationEvent[blockHash] = event
            hashToCreationTimestamp[blockHash] = timestamp
        if event == ProposerBlockConfirmed:
            if hashToConfirmationTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToConfirmationTimestamp[blockHash] = timestamp
        if event == ProposerBlockRolledBack:
            if hashToRollbackTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToRollbackTimestamp[blockHash] = timestamp

f.close()

In [13]:
df = pd.DataFrame(columns=['Block Hash','Block Number','Timestamp - Creation','Event','Timestamp - Confirmation','Timestamp - Rollback'])

for index,key in enumerate(hashToNumber.keys()):
    df.loc[index] = [key,
        int(hashToNumber.get(key,'')),
        hashToCreationTimestamp.get(key,''),
        hashToCreationEvent.get(key,''),
        hashToConfirmationTimestamp.get(key,''),
        hashToRollbackTimestamp.get(key,'')]

df.sort_values('Block Number')

,Block Hash,Block Number,Timestamp - Creation,Event,Timestamp - Confirmation,Timestamp - Rollback
0,0x9579d55926e3391f152be728b158f5f494e251e017e6...,1,1.638987e+12,ProposerBlockMined,1638990776118.0,
1,0xae693736d5aa7524db8185743f3b86e82fac8c94bb66...,2,1.638987e+12,ProposerBlockMined,1638990842611.0,
2,0xc82029a534fd6076b8097b2bfdbe45a55931e0c944e9...,3,1.638988e+12,ProposerBlockMined,1638990954505.0,
3,0xed3182fbf3f02b63dc5660b0dd13aa9524f9da25f080...,4,1.638988e+12,ProposerBlockMined,1638990954512.0,
4,0x415863ec96f522060562e311c6a14bf4d3cbe7848c0f...,5,1.638988e+12,ProposerBlockMined,1638990954519.0,
...,...,...,...,...,...,...
162,0xcd2f2e9f56e7a7e1ebb6c66041f5c8925aadfcf969e1...,163,1.638995e+12,ProposerBlockMined,,
163,0x78103819104558f307e432a94afd033afb8207498198...,164,1.638995e+12,ProposerBlockMined,,
164,0xa9b57317b679fa6290aa307499662e529fe36f39132b...,165,1.638995e+12,ProposerBlockMined,,
165,0x948496dfa64c1d1f1221ff7f9be0a464628b2ebd8ebb...,166,1.638995e+12,ProposerBlockMined,,


In [14]:
df['Latency'] = df[df['Timestamp - Confirmation']!='']['Timestamp - Confirmation']-df[df['Timestamp - Confirmation']!='']['Timestamp - Creation']
df['Mining Time'] = df['Timestamp - Creation'].diff()
df['Confirmation Depth']=df['Latency']/df['Mining Time']

In [15]:
print("Mean Latency =",df['Latency'].dropna().mean())
print("Median Latency =",df['Latency'].dropna().median())
print("Mean Mining Time =",df['Mining Time'].dropna().mean())
print("Median Mining Time =",df['Mining Time'].dropna().median())
print("Mean Confirmation Depth =",df['Confirmation Depth'].dropna().mean())
print("Median Confirmation Depth =",df['Confirmation Depth'].dropna().median())

Mean Latency = 3038678.786407767
Median Latency = 3016869.0
Mean Mining Time = 45306.03614457831
Median Mining Time = 28941.5
Mean Confirmation Depth = 356.73973797754076
Median Confirmation Depth = 109.52248782055852


In [16]:
creationTimestamps = df['Timestamp - Creation']
confirmationTimestamps = df[df['Timestamp - Confirmation']!='']['Timestamp - Confirmation']
insertionPoints = pd.DataFrame(creationTimestamps.searchsorted(value=confirmationTimestamps),columns=['Insertion Points'])
cdArray = insertionPoints['Insertion Points'] - insertionPoints['Insertion Points'].index
print("Trial Mean Confirmation Depth =",cdArray.mean())
print("Trial Median Confirmation Depth =",cdArray.median())

Trial Mean Confirmation Depth = 67.4368932038835
Trial Median Confirmation Depth = 67.0
